# Solvers

A constraints-based reconstruction and analysis model for biological systems is actually just an application of a class of discrete optimization problems typically solved with [linear, mixed integer](https://en.wikipedia.org/wiki/Linear_programming) or [quadratic programming](https://en.wikipedia.org/wiki/Quadratic_programming) techniques. Cobrapy does not implement any algorithms to find solutions to such problems but rather creates an biologically motivated abstraction to these techniques to make it easier to think of how metabolic systems work without paying much attention to how that formulates to an optimization problem.

The actual solving is instead done by tools such as the free software [glpk](https://www.gnu.org/software/glpk/) or commercial tools [gurobi](http://www.gurobi.com/) and [cplex](https://www-01.ibm.com/software/commerce/optimization/cplex-optimizer/) which are all made available as a common programmers interface via the [optlang](https://github.com/biosustain/optlang) package.

When you have defined your model, you can switch solver backend by simply assigning to the `model.solver` property.

In [1]:
import cobra.test
model = cobra.test.create_test_model('textbook')

/home/moritz/.virtualenvs/cobra/lib/python3.5/site-packages/optlang/gurobi_interface.py:26: UserWarning: Be careful! The GUROBI interface is still under construction ...
  warn("Be careful! The GUROBI interface is still under construction ...")


In [2]:
model.solver = 'glpk'
# or if you have cplex installed
model.solver = 'cplex'

For information on how to configure and tune the solver, please see the [documentation for optlang project](http://optlang.readthedocs.io) and note that `model.solver` is simply an object optlang of class `Model`.

In [3]:
type(model.solver)

optlang.cplex_interface.Model

## Internal solver interfaces

Cobrapy also contains its own solver interfaces but these are now deprecated and will be removed completely in the near future. For documentation of how to use these, please refer to [older documentation](http://cobrapy.readthedocs.io/en/0.5.11/).